模型复杂度分析 \

给定输入尺寸 inputs = torch.randn((1, 3, 10, 10))，和一个卷积层 conv = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)，那么它输出的特征图尺寸为 (1, 10, 8, 8)，则它的浮点运算量是 17280 = 10*8*8*3*3*3（1088 表示输出的特征图大小、333 表示每一个输出需要的计算量）、激活量是 640 = 10*8*8、参数量是 280 = 3*10*3*3 + 10（3103*3 表示权重的尺寸、10 表示偏置值的尺寸）

激活量是指产生了多少个值

In [1]:
#继承nn.Module的模型
from torch import nn

from mmengine.analysis import get_model_complexity_info


# 以字典的形式返回分析结果，包括:
# ['flops', 'flops_str', 'activations', 'activations_str', 'params', 'params_str', 'out_table', 'out_arch']
class InnerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        return self.fc1(self.fc2(x))


class TestNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 10)
        self.fc2 = nn.Linear(10, 10)
        self.inner = InnerNet()

    def forward(self, x):
        return self.fc1(self.fc2(self.inner(x)))


input_shape = (1, 10)
model = TestNet()

analysis_results=get_model_complexity_info(model,input_shape)

/root/anaconda3/envs/mmyolo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


输出的结果有七个字段\
flops: flop 的总数, 例如, 1000, 1000000\
flops_str: 格式化的字符串, 例如, 1.0G, 1.0M\
params: 全部参数的数量, 例如, 1000, 1000000\
params_str: 格式化的字符串, 例如, 1.0K, 1M\
activations: 激活量的总数, 例如, 1000, 1000000\
activations_str: 格式化的字符串, 例如, 1.0G, 1M\
out_table: 以表格形式打印相关信息

In [2]:
print(analysis_results['out_table'])


+---------------------+----------------------+--------+--------------+
| module              | #parameters or shape | #flops | #activations |
+---------------------+----------------------+--------+--------------+
| model               | 0.44K                | 0.4K   | 40           |
|  fc1                |  0.11K               |  100   |  10          |
|   fc1.weight        |   (10, 10)           |        |              |
|   fc1.bias          |   (10,)              |        |              |
|  fc2                |  0.11K               |  100   |  10          |
|   fc2.weight        |   (10, 10)           |        |              |
|   fc2.bias          |   (10,)              |        |              |
|  inner              |  0.22K               |  0.2K  |  20          |
|   inner.fc1         |   0.11K              |   100  |   10         |
|    inner.fc1.weight |    (10, 10)          |        |              |
|    inner.fc1.bias   |    (10,)             |        |              |
|   i

In [3]:
print(analysis_results['out_arch'])


TestNet(
  #params: 0.44K, #flops: 0.4K, #acts: 40
  (fc1): Linear(
    in_features=10, out_features=10, bias=True
    #params: 0.11K, #flops: 100, #acts: 10
  )
  (fc2): Linear(
    in_features=10, out_features=10, bias=True
    #params: 0.11K, #flops: 100, #acts: 10
  )
  (inner): InnerNet(
    #params: 0.22K, #flops: 0.2K, #acts: 20
    (fc1): Linear(
      in_features=10, out_features=10, bias=True
      #params: 0.11K, #flops: 100, #acts: 10
    )
    (fc2): Linear(
      in_features=10, out_features=10, bias=True
      #params: 0.11K, #flops: 100, #acts: 10
    )
  )
)


In [4]:
#继承mmengine.models.BaseModel的模型
#没感觉用法上有什么区别
import torch.nn.functional as F
import torchvision
from mmengine.model import BaseModel
from mmengine.analysis import get_model_complexity_info


class MMResNet50(BaseModel):
    def __init__(self):
        super().__init__()
        self.resnet = torchvision.models.resnet50()

    def forward(self, imgs, labels=None, mode='tensor'):
        x = self.resnet(imgs)
        if mode == 'loss':
            return {'loss': F.cross_entropy(x, labels)}
        elif mode == 'predict':
            return x, labels
        elif mode == 'tensor':
            return x


input_shape = (3, 224, 224)
model = MMResNet50()

analysis_results = get_model_complexity_info(model, input_shape)

print("Model Flops:{}".format(analysis_results['flops_str']))
# Model Flops:4.145G
print("Model Parameters:{}".format(analysis_results['params_str']))
# Model Parameters:25.557M

08/31 07:19:54 - mmengine - WARNING - Unsupported operator aten::add_ encountered 69 time(s)
08/31 07:19:54 - mmengine - WARNING - Unsupported operator aten::max_pool2d encountered 1 time(s)
08/31 07:19:54 - mmengine - WARNING - The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
data_preprocessor
08/31 07:19:54 - mmengine - WARNING - Unsupported operator aten::batch_norm encountered 53 time(s)
08/31 07:19:54 - mmengine - WARNING - Unsupported operator aten::adaptive_avg_pool2d encountered 1 time(s)


Model Flops:4.145G
Model Parameters:25.557M
